# WordPress Development Workflow
## Správné nastavení DEV a PROD prostředí

**Základní filozofie:** Nikdy neupravujeme produkci přímo. Vždy pracujeme přes DEV → Git → PROD workflow.

## Krok 0: Základní myšlenka

Představ si to jako práci s dokumentem:
- **"Živá" verze** na firemním disku = **PRODUKCE** (hosting)
- **Kopie** u tebe na notebooku = **DEV** (Docker)

### Zlaté pravidlo:
1. Nikdy neupravuješ živou verzi
2. Vždy si ji nejdřív zkopíruješ k sobě
3. U sebe děláš změny
4. Až jsi hotový, nahradíš živou verzi tou svojí upravenou

## Krok 1: Vytvoření identických prostředí (Startovní čára)

**Cíl:** Obě prostředí musí být na začátku co nejvíce identická.

### 1.1 Čistá instalace na PRODUKCI

In [ ]:
# Na hostingu nainstaluj čistý WordPress
# - Použij autoinstalátor hostingu
# - Nezatěžuj se obsahem (žádné stránky, články)
# - Toto je tvůj "master" obraz

echo "PRODUKCE: Čistá instalace WordPressu dokončena"
echo "URL: http://soustonamiru.cz"
echo "Databáze: soustonamirucz"
echo "Uživatel: soustonamiruc001"
echo "Heslo: 55e8rVMc"
echo "Host: 127.0.0.1:3306"

### 1.2 Stažení souborů z produkce na DEV

In [ ]:
# Stáhni všechny soubory z produkce
cd /opt/www/soustonamiru.cz

# Zálohuj současný stav
mv wp-content wp-content-backup-$(date +%Y%m%d) 2>/dev/null || true

# Stáhni čerstvé soubory z produkce
sshpass -p "qO0DvDnUPxgrLUGp@N" scp -r soustonamiru_cz@debbie.thinline.cz:soustonamiru.cz/* .

echo "Soubory staženy z produkce"

### 1.3 Export databáze z produkce

In [ ]:
# Export produkční databáze
sshpass -p "qO0DvDnUPxgrLUGp@N" ssh soustonamiru_cz@debbie.thinline.cz \
"mysqldump -h 127.0.0.1 -u soustonamiruc001 -p55e8rVMc soustonamirucz > prod_backup_$(date +%Y%m%d).sql"

# Stáhni si backup
sshpass -p "qO0DvDnUPxgrLUGp@N" scp soustonamiru_cz@debbie.thinline.cz:prod_backup_*.sql ./

echo "Databáze exportována z produkce"

### 1.4 Import databáze do DEV prostředí

In [ ]:
# Import do lokální databáze
# Nejdříve zjisti název MariaDB kontejneru
MARIADB_CONTAINER=$(docker ps --format "table {{.Names}}" | grep mariadb)

# Import databáze
docker exec -i $MARIADB_CONTAINER \
mysql -u root -proot_password wordpress < prod_backup_$(date +%Y%m%d).sql

echo "Databáze importována do DEV"

### 1.5 Úprava wp-config.php pro DEV

In [ ]:
# Vytvoř wp-config.php pro DEV prostředí
cat > wp-config-dev.php << 'EOF'
<?php
// DEV prostředí
define('DB_NAME', 'wordpress');
define('DB_USER', 'wordpress');
define('DB_PASSWORD', 'wordpress');
define('DB_HOST', 'mariadb');
define('DB_CHARSET', 'utf8mb4');
define('DB_COLLATE', '');

// Zkopíruj soli z produkční wp-config.php
define('AUTH_KEY',         'put your unique phrase here');
define('SECURE_AUTH_KEY',  'put your unique phrase here');
define('LOGGED_IN_KEY',    'put your unique phrase here');
define('NONCE_KEY',        'put your unique phrase here');
define('AUTH_SALT',        'put your unique phrase here');
define('SECURE_AUTH_SALT', 'put your unique phrase here');
define('LOGGED_IN_SALT',   'put your unique phrase here');
define('NONCE_SALT',       'put your unique phrase here');

$table_prefix = 'wp_';
define('WP_DEBUG', true);
define('WP_DEBUG_LOG', true);

if ( ! defined( 'ABSPATH' ) ) {
	define( 'ABSPATH', __DIR__ . '/' );
}
require_once ABSPATH . 'wp-settings.php';
EOF

# Použij DEV konfiguraci
cp wp-config-dev.php wp-config.php

echo "wp-config.php upraven pro DEV"

### 1.6 Změna URL v databázi

In [ ]:
# Vytvoř skript pro změnu URL
cat > change-urls.php << 'EOF'
<?php
require_once('wp-config.php');

// Připoj se k databázi
$mysqli = new mysqli(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME);

if ($mysqli->connect_error) {
    die('Chyba připojení: ' . $mysqli->connect_error);
}

// Změň URL z produkce na DEV
$old_url = 'http://soustonamiru.cz';
$new_url = 'http://localhost:8085';

$queries = [
    "UPDATE wp_options SET option_value = '$new_url' WHERE option_name = 'home'",
    "UPDATE wp_options SET option_value = '$new_url' WHERE option_name = 'siteurl'",
    "UPDATE wp_posts SET post_content = REPLACE(post_content, '$old_url', '$new_url')",
    "UPDATE wp_comments SET comment_content = REPLACE(comment_content, '$old_url', '$new_url')"
];

foreach ($queries as $query) {
    if ($mysqli->query($query)) {
        echo "Úspěch: $query\n";
    } else {
        echo "Chyba: " . $mysqli->error . "\n";
    }
}

$mysqli->close();
echo "URL změněny z produkce na DEV\n";
?>
EOF

# Spusť změnu URL
php change-urls.php

echo "URL změněny pro DEV prostředí"

## ✅ VÝSLEDEK KROKU 1

Nyní máš **dvě naprosto identické kopie webu**:
- **PRODUKCE:** http://soustonamiru.cz
- **DEV:** http://localhost:8085

**Jsi na startovní čáře!** 🏁

## Krok 2: Správa pluginů a témat (Best Practice)

**ZLATÉ PRAVIDLO:** Všechny změny kódu (pluginy, témata) děláš POUZE na DEV prostředí!

### 2.1 Instalace pluginu na DEV

In [ ]:
# Příklad: Instalace Wordfence pluginu na DEV
echo "1. Přihlaš se do administrace: http://localhost:8085/wp-admin"
echo "2. Jdi do Pluginy > Přidat nový"
echo "3. Vyhledej 'Wordfence' a nainstaluj"
echo "4. Aktivuj a nastav plugin"
echo ""
echo "VÝSLEDEK: Tvůj lokální web má nový plugin. Produkce o něm zatím neví."

# Po instalaci zkontroluj nové soubory
ls -la wp-content/plugins/ | grep wordfence

### 2.2 Commitni změny do Git

In [ ]:
# Přidej nové soubory do Git
git add wp-content/plugins/wordfence/
git add wp-content/uploads/  # případné nové uploadované soubory

# Commit s popisnou zprávou
git commit -m "Přidán Wordfence security plugin"

# Push do repozitáře
git push origin main

echo "Plugin přidán do Git repozitáře"

## Krok 3: Přenos změn z DEV na PRODUKCI

**Magie Git deploymetu:** Jak dostat změny na produkci, aniž bys přepsal obsah?

### 3.1 Oddělíme soubory vs. databázi

- **Soubory** (pluginy, témata) → řeší **Git**
- **Nastavení/obsah** (databáze) → řešíme **selektivní export/import**

### 3.2 Deployment skript

In [ ]:
# Vytvoř deployment skript
mkdir -p Deployment
cat > Deployment/deploy.sh << 'EOF'
#!/bin/bash

echo "🚀 Zahajuji deployment na produkci..."

# 1. Backup produkce
echo "📦 Zálohuji produkční soubory..."
sshpass -p "qO0DvDnUPxgrLUGp@N" ssh soustonamiru_cz@debbie.thinline.cz \
"cd soustonamiru.cz && tar -czf backup_$(date +%Y%m%d_%H%M%S).tar.gz wp-content/"

# 2. Pull nových změn
echo "⬇️ Stahuji nové změny z Git..."
sshpass -p "qO0DvDnUPxgrLUGp@N" ssh soustonamiru_cz@debbie.thinline.cz \
"cd soustonamiru.cz && git pull origin main"

# 3. Zkontroluj wp-config.php
echo "⚙️ Kontroluji wp-config.php..."
sshpass -p "qO0DvDnUPxgrLUGp@N" ssh soustonamiru_cz@debbie.thinline.cz \
"cd soustonamiru.cz && cp wp-config-prod.php wp-config.php"

# 4. Nastavení práv
echo "🔐 Nastavuji správná práva..."
sshpass -p "qO0DvDnUPxgrLUGp@N" ssh soustonamiru_cz@debbie.thinline.cz \
"cd soustonamiru.cz && chmod -R 755 wp-content/"

echo "✅ Deployment dokončen!"
echo "🌐 Zkontroluj: http://soustonamiru.cz"
EOF

chmod +x Deployment/deploy.sh
echo "Deployment skript vytvořen"

### 3.3 Spuštění deploymetu

In [ ]:
# Spusť deployment
./Deployment/deploy.sh

echo "🎉 Plugin je nyní aktivní na produkci!"

## Krok 4: Databázové migrace (Pokročilé)

**Problém:** Plugin potřebuje také nastavení z databáze.

### 4.1 Selektivní export nastavení pluginu

In [ ]:
# Export pouze nastavení pluginu (ne obsah!)
cat > export-plugin-settings.php << 'EOF'
<?php
require_once('wp-config.php');

$mysqli = new mysqli(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME);

// Export pouze options tabulky s nastavením Wordfence
$result = $mysqli->query(
    "SELECT * FROM wp_options WHERE option_name LIKE 'wordfence%'"
);

echo "-- Nastavení Wordfence pluginu\n";
while ($row = $result->fetch_assoc()) {
    $name = $mysqli->real_escape_string($row['option_name']);
    $value = $mysqli->real_escape_string($row['option_value']);
    
    echo "INSERT INTO wp_options (option_name, option_value, autoload) ";
    echo "VALUES ('$name', '$value', '{$row['autoload']}') ";
    echo "ON DUPLICATE KEY UPDATE option_value='$value';\n";
}
?>
EOF

# Vygeneruj SQL soubor s nastavením
php export-plugin-settings.php > wordfence-settings.sql

echo "Nastavení pluginu exportováno"

### 4.2 Import nastavení na produkci

In [ ]:
# Nahraj a importuj nastavení
sshpass -p "qO0DvDnUPxgrLUGp@N" scp wordfence-settings.sql \
soustonamiru_cz@debbie.thinline.cz:soustonamiru.cz/

# Import do produkční databáze
sshpass -p "qO0DvDnUPxgrLUGp@N" ssh soustonamiru_cz@debbie.thinline.cz \
"cd soustonamiru.cz && mysql -h 127.0.0.1 -u soustonamiruc001 -p55e8rVMc soustonamirucz < wordfence-settings.sql"

echo "Nastavení pluginu importováno na produkci"

## 🎯 Výsledný workflow

### Každodenní práce:

1. **Změny POUZE na DEV** (localhost:8085)
2. **Commit do Git** (`git add . && git commit -m "popis změny"`)
3. **Push** (`git push origin main`)
4. **Deployment** (`./Deployment/deploy.sh`)
5. **Test na produkci** (https://soustonamiru.cz)

### Výhody:
- ✅ Nikdy neztratíš práci
- ✅ Můžeš vrátit jakoukoliv změnu
- ✅ Testování před nasazením
- ✅ Automatizovaný deployment
- ✅ Bezpečnost produkce

## 🎯 Výsledný workflow

### Každodenní práce:

1. **Změny POUZE na DEV** (http://localhost:8085)
2. **Commit do Git** (`git add . && git commit -m "popis změny"`)
3. **Push** (`git push origin main`)
4. **Deployment** (`./Deployment/deploy.sh`)
5. **Test na produkci** (http://soustonamiru.cz)

### Výhody:
- ✅ Nikdy neztratíš práci
- ✅ Můžeš vrátit jakoukoliv změnu
- ✅ Testování před nasazením
- ✅ Automatizovaný deployment
- ✅ Bezpečnost produkce

## 🚨 Důležité poznámky

### Co NIKDY nedělat:
- ❌ Neupravuj soubory přímo na produkci
- ❌ Neinstaluj pluginy přímo na produkci
- ❌ Nepřepisuj celou databázi na produkci

### Co sledovat:
- 📁 **Soubory:** všechny jdou přes Git
- 🗄️ **Obsah:** (články, stránky) vzniká na produkci
- ⚙️ **Nastavení:** selektivní migrace podle potřeby

### V případě problémů:
- 🔄 Rollback: `git revert <commit>`
- 💾 Restore: ze záložních souborů
- 🔍 Debug: WP_DEBUG = true na DEV